### 编写yaml配置文件

对于生成图像数据评估，可编写如下格式的yaml文件，其中data下的配置用于指定数据集的路径和相关信息，scorers下的配置用于指定您想使用的评估指标。
```yaml
model_cache_path: '../ckpt' # Path to cache models
num_workers: 2

data:
  image:
    meta_data_path: "../data/image_data.jsonl" # 元数据的存放位置
    data_path: "../data/images" # 图像数据的存放位置
    image_key: 'image' # 元数据中图像路径（或图像名）对应的键
    id_key: 'id' # 元数据中id对应的键
    formatter: 'GenImageFormatter' # 生成图片数据固定使用GenImageFormatter

    # 可选，对于一些指标需要参考数据，例如FID和KID，必须在这里指定。对于IS，默认为None
    ref_meta_data_path: "../data/ref_image_data.jsonl" 
    ref_data_path: "../data/images"

scorers:
  ISScorer:
    batch_size: 32
    resize: True
    splits: 10
    device: "cpu"
  FIDScorer:
    batch_size: 50
    dims: 2048
    model: https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth
    device: "cpu"
  KIDScorer:
    batch_size: 50
    dims: 2048
    model:  inception
    device: "cpu"
```

以上配置文件对应的元数据格式如下(以LLaVA pretrain数据为例)：
```json
{"id": "003633483", "text": "a logo for district nurse with a rose design mugs", "image": "images/00011/000116355.jpg"}
{"id": "003241608", "text": "a playmobil police activity calendar", "image": "images/00025/000256174.jpg"}
{"id": "004275678", "text": "the front cover of squares three theories by john e stauffer", "image": "images/00013/000137376.jpg"}
{"id": "004551653", "text": "camel special lights cigarettes box", "image": "images/00035/000359122.jpg"}
{"id": "001114792", "text": "a barn in the rain iphone case", "image": "images/00039/000395694.jpg"}
```
ref_image_data.json的格式与image_data.json相同。

##### 数据加载
对于图片数据集，支持json、jsonl等格式。

对于fid和kid，需要传入两个数据集，来计算两个数据集间的分数。

对于is，只需要传入需要评估的数据集。

In [1]:
import sys
import os
datagym_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..')) 
sys.path.insert(0, datagym_path)
import datagym
from datagym.data.image_dataset import jsonImageDataset

image_folder_path = "../data/images"
image_dataset1 = jsonImageDataset("../data/image_data.jsonl", image_folder_path)
image_dataset2 = jsonImageDataset("../data/ref_image_data.jsonl", image_folder_path)
print(len(image_dataset1), len(image_dataset2))

750 750


##### IS使用示例
IS（Inception Score）是一种评估图像生成模型性能的方法。以下代码创建了IS评分器，并对指定的数据集进行了评分。配置yaml文件中的scorers如下，并打印输出。
```yaml
scorers:
  ISScorer:
    batch_size: 32
    resize: True
    splits: 10
    device: "cpu"
```

In [2]:
!python run_images.py --config ../../configs/is_score.yaml

/mnt/petrelfs/chenjingzhou/anaconda3/envs/dg2/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/mnt/petrelfs/chenjingzhou/anaconda3/envs/dg2/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


/mnt/petrelfs/chenjingzhou/lz/DataGym/datagym/Eval/image/is_scorer.py:64: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x).data.cpu().numpy()


{}
{'ISScorer': (4.777031371280047, 0.25132291953034813)}


##### FID使用示例

FID（Frechet Inception Distance）是通过比较真实图像和生成图像的特征分布差异来评估图像质量的方法。以下代码演示了如何初始化FID评分器，并对两个数据集进行评分。配置yaml文件中的scorers如下，并打印输出。
```yaml
scorers:
  FIDScorer:
    batch_size: 50
    dims: 2048
    model: https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth
    device: "cpu"
```

In [3]:
!python run_images.py --config ../../configs/fid_score.yaml

2048



  0%|                                                    | 0/15 [00:00<?, ?it/s]


  7%|██▉                                         | 1/15 [00:00<00:12,  1.16it/s]


 20%|████████▊                                   | 3/15 [00:00<00:03,  3.76it/s]


 33%|██████████████▋                             | 5/15 [00:01<00:01,  6.32it/s]

...


 73%|███████████████████████████████▌           | 11/15 [00:00<00:00, 17.47it/s]


 87%|█████████████████████████████████████▎     | 13/15 [00:00<00:00, 17.90it/s]


100%|███████████████████████████████████████████| 15/15 [00:00<00:00, 18.21it/s]


100%|███████████████████████████████████████████| 15/15 [00:00<00:00, 15.39it/s]


{}
{'FIDScorer': 62.86046101873319}


##### KID使用示例

KID（Kernel Inception Distance）与FID类似，但使用不同的统计方法来评估分布差异。以下代码同样展示了如何进行KID评分的计算。配置yaml文件中的scorers如下，并打印输出。
```yaml
scorers:
  KIDScorer:
    batch_size: 50
    dims: 2048
    model:  inception
    device: "cpu"
```

In [4]:
!python run_images.py --config ../../configs/kid_score.yaml


  0%|                                                    | 0/15 [00:00<?, ?it/s]


  7%|██▉                                         | 1/15 [00:00<00:06,  2.08it/s]


 20%|████████▊                                   | 3/15 [00:00<00:02,  5.65it/s]


 33%|██████████████▋                             | 5/15 [00:00<00:01,  8.33it/s]


 47%|████████████████████▌                       | 7/15 [00:00<00:00, 10.32it/s]

...


MMD:  97%|██████████████████████▎| 97/100 [00:32<00:00,  3.15it/s, mean=0.00112]


MMD:  98%|██████████████████████▌| 98/100 [00:33<00:00,  3.16it/s, mean=0.00111]


MMD:  99%|██████████████████████▊| 99/100 [00:33<00:00,  3.17it/s, mean=0.00112]


MMD: 100%|██████████████████████| 100/100 [00:33<00:00,  2.96it/s, mean=0.00112]


{}
{'KIDScorer': (0.0011184606246640437, 0.0002335252548380459)}


##### 具体应用
该部分详细介绍 FID, KID 和 IS 的具体应用，并通过这些指标来评估和比较不同生成模型在图像生成质量上的表现。使用了四种模型：flux-dev, flux-schnell, stable-diffusion-3-medium 和 sdxl，对在 LLaVA Pretrain 数据集上随机选取的500个 image-caption 对进行测试。每个模型根据给定的 caption 生成相应的图片，并通过上述三个指标对生成图片的质量进行全面评估。下面提供模型评估结果：

| 模型名称                  | Inception Score (IS)               | Fréchet Inception Distance (FID) | Kernel Inception Distance (KID)               |
|-------------------------|-----------------------------------|---------------------------------|-----------------------------------------------|
| flux-dev                | 7.195 ± 0.809                     | 101.572                         | 0.00903 ± 0.00069                             |
| flux-schnell            | 6.193 ± 0.546                     | 102.739                         | 0.00667 ± 0.00055                             |
| stable-diffusion-3-medium | 6.740 ± 0.582                     | 100.235                         | 0.00609 ± 0.00056                             |
| sdxl                    | 6.809 ± 0.994                     | 112.807                         | 0.01051 ± 0.00065                             |